In [1]:
import pandas as pd
import os
import numpy as np
import time
import os
import csv
import requests
import webbrowser
from datetime import datetime
from datetime import timedelta


In [2]:
## On stock ici la clé API

API_KEY = '69075b791296e1.23111363'
symbol = 'GME'

In [3]:
url = ("https://eodhd.com/api/mp/unicornbay/options/eod?"
    "filter[underlying_symbol]=NVDA"
    "&filter[exp_date_from]=2024-01-01"
    "&filter[exp_date_to]=2025-12-31"
    f"&sort=-exp_date&api_token={API_KEY}")

response = requests.get(url)
data = response.json()

# Convert to dataframe
df = pd.DataFrame(data["data"])

# Exploser la colonne attributes en colonnes séparées
attr_df = pd.json_normalize(df["attributes"])

# Fusionner avec les colonnes principales (id, type, etc)
df_clean = pd.concat([df[["id", "type"]], attr_df], axis=1)


In [4]:
df_clean.shape

(1000, 45)

In [5]:
webbrowser.open_new(url)

True

In [6]:
df_clean

,id,type,contract,underlying_symbol,exp_date,expiration_type,type,strike,exchange,currency,...,theoretical,delta,gamma,theta,vega,rho,tradetime,vol_oi_ratio,dte,midpoint
0,NVDA251226P00390000-2025-12-25,options-eod,NVDA251226P00390000,NVDA,2025-12-26,weekly,put,390.0,NASDAQ,USD,...,201.50,-0.991732,0.000421,-0.290025,0.003134,-0.008423,2025-12-24,0.00,2,201.50
1,NVDA251226P00380000-2025-12-25,options-eod,NVDA251226P00380000,NVDA,2025-12-26,weekly,put,380.0,NASDAQ,USD,...,191.43,-0.996012,0.000251,-0.129605,0.001633,-0.006381,None,0.00,2,191.43
2,NVDA251226P00370000-2025-12-25,options-eod,NVDA251226P00370000,NVDA,2025-12-26,weekly,put,370.0,NASDAQ,USD,...,181.43,-0.995924,0.000265,-0.127775,0.001665,-0.006245,None,0.00,2,181.43
3,NVDA251226P00360000-2025-12-25,options-eod,NVDA251226P00360000,NVDA,2025-12-26,weekly,put,360.0,NASDAQ,USD,...,171.43,-0.995824,0.000281,-0.125901,0.001701,-0.006110,None,0.00,2,171.43
4,NVDA251226P00350000-2025-12-25,options-eod,NVDA251226P00350000,NVDA,2025-12-26,weekly,put,350.0,NASDAQ,USD,...,161.43,-0.995712,0.000299,-0.123977,0.001742,-0.005975,None,0.00,2,161.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NVDA251226C00165000-2025-12-17,options-eod,NVDA251226C00165000,NVDA,2025-12-26,weekly,call,165.0,NASDAQ,USD,...,8.05,0.722838,0.030200,-0.217846,0.089906,0.028483,2025-12-17,1.05,9,7.95
996,NVDA251226C00162500-2025-12-17,options-eod,NVDA251226C00162500,NVDA,2025-12-26,weekly,call,162.5,NASDAQ,USD,...,9.43,0.834470,0.026677,-0.142049,0.066769,0.032849,2025-12-17,0.33,9,9.43
997,NVDA251226C00160000-2025-12-17,options-eod,NVDA251226C00160000,NVDA,2025-12-26,weekly,call,160.0,NASDAQ,USD,...,11.90,0.858615,0.020373,-0.150259,0.060144,0.033256,2025-12-17,0.52,9,11.90
998,NVDA251226C00157500-2025-12-17,options-eod,NVDA251226C00157500,NVDA,2025-12-26,weekly,call,157.5,NASDAQ,USD,...,14.13,0.900784,0.015448,-0.123330,0.046838,0.034485,2025-12-17,1.79,9,14.13


In [8]:
df_clean.to_csv('chain_test.csv', sep =',')

# On va récupérer ici les différentes dates d'expirations

In [5]:

def fetch_options(start, end):
    url = (
        f"https://eodhd.com/api/mp/unicornbay/options/eod"
        f"?filter[underlying_symbol]={symbol}"
        f"&filter[exp_date_from]={start}"
        f"&filter[exp_date_to]={end}"
        f"&api_token={API_KEY}&fmt=json"
    )
    r = requests.get(url)
    r.raise_for_status()
    return r.json().get("data", [])

# Découper en trimestres pour éviter la limite
periods = [
    ("2024-01-01", "2024-03-31"),
    ("2024-04-01", "2024-06-30"),
    ("2024-07-01", "2024-09-30"),
    ("2024-10-01", "2024-12-31"),
    ("2025-01-01", "2025-03-31"),
    ("2025-04-01", "2025-06-30"),
    ("2025-07-01", "2025-09-30"),
    ("2025-10-01", "2025-12-31")
]

all_rows = []
for start, end in periods:
    all_rows += fetch_options(start, end)

df = pd.DataFrame([row["attributes"] for row in all_rows])
expirations = sorted(df["exp_date"].dropna().unique())

print(f"Dates d'expiration pour {symbol}:")
for d in expirations:
    print(d)


Dates d'expiration pour GME:
2024-03-28
2024-06-28
2024-09-27
2024-12-27
2025-03-28
2025-06-27
2025-09-26
2025-12-12
2025-12-19
2025-12-26


# On remarque ici que l'on peut récupérer toutes les différentes dates d'expiration de contrats pour un certain sous jacent. On va maintenant récupérer tous les contrats pour une certaine échéance passée:


In [7]:
def option_chain(date_init, exp_date, API_KEY, symbol):

   
    
    url= ("https://eodhd.com/api/mp/unicornbay/options/eod?"
        f"filter[underlying_symbol]={symbol}"
        "&filter[exp_date]=2025-09-26"
        "&sort=exp_date&page[limit]=1000"
        "&page[offset]=2000"
        f"&sort=-exp_date&api_token={API_KEY}")
    
    response = requests.get(url)
    data = response.json()
    
    # Convert to dataframe
    df_2025_09_26 = pd.DataFrame(data["data"])
    
    # Exploser la colonne attributes en colonnes séparées
    attr_df_2025_09_26 = pd.json_normalize(df_2025_09_26["attributes"])
    
    # Fusionner avec les colonnes principales (id, type, etc)
    df_2025_09_26_clean = pd.concat([df_2025_09_26[["id", "type"]], attr_df_2025_09_26], axis=1)


In [8]:
df_2025_09_26_clean

NameError: name 'df_2025_09_26_clean' is not defined

On est ici bridé par la pagination qui limite le nombre de lignes à 1000, on va donc contourner ce probleme

In [35]:
webbrowser.open_new(url_2025_09_26)

True

In [9]:
exp_date = '2025-09-19'

In [7]:
def historical_option_chain_by_tradetime(start_date, exp_date, API_KEY, symbol):
    """
    Récupère les données d'options jour par jour pour une date d'expiration donnée.
    """
    date_format = "%Y-%m-%d"
    current_date = datetime.strptime(start_date, date_format)
    end_date = datetime.strptime(exp_date, date_format)

    all_dfs = []

    while current_date <= end_date:
        trade_date = current_date.strftime(date_format)
        print(f"Téléchargement pour {symbol} | expiration: {exp_date} | jour: {trade_date}")

        offset = 0
        limit = 1000

        while True:
            url = (
                "https://eodhd.com/api/mp/unicornbay/options/eod?"
                f"filter[underlying_symbol]={symbol}"
                f"&filter[exp_date_eq]={exp_date}"
                f"&filter[tradetime_eq]={trade_date}"
                f"&page[limit]={limit}&page[offset]={offset}"
                f"&api_token={API_KEY}"
            )

            response = requests.get(url)
            if response.status_code != 200:
                print(f" Erreur API pour {trade_date}, offset={offset} : {response.status_code}")
                break

            data = response.json().get("data", [])
            if not data:
                break

            df_raw = pd.DataFrame(data)
            df_attr = pd.json_normalize(df_raw["attributes"])
            df_clean = pd.concat([df_raw[["id", "type"]], df_attr], axis=1)
            df_clean["data_retrieved_for"] = trade_date  # utile pour suivi

            all_dfs.append(df_clean)

            if len(data) < limit:
                break

            offset += limit
            time.sleep(0.2)

        current_date += timedelta(days=1)

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
    else:
        final_df = pd.DataFrame()

    return final_df


In [8]:
df_nvda_2025_12_19 = historical_option_chain_by_tradetime(
    start_date="2025-01-01",
    exp_date="2025-09-26",
    API_KEY = API_KEY,
    symbol = 'NVDA')

Téléchargement pour NVDA | expiration: 2025-09-26 | jour: 2025-01-01
Téléchargement pour NVDA | expiration: 2025-09-26 | jour: 2025-01-02
Téléchargement pour NVDA | expiration: 2025-09-26 | jour: 2025-01-03
Téléchargement pour NVDA | expiration: 2025-09-26 | jour: 2025-01-04
Téléchargement pour NVDA | expiration: 2025-09-26 | jour: 2025-01-05
Téléchargement pour NVDA | expiration: 2025-09-26 | jour: 2025-01-06
Téléchargement pour NVDA | expiration: 2025-09-26 | jour: 2025-01-07


KeyboardInterrupt: 

In [12]:
df_nvda_2025_09_26 = df_nvda_2025_12_19

In [13]:
df_nvda_2025_09_26.to_csv('df_gamestop_2025_09_26.csv', sep = ';',decimal='.' , index = False)

# On a ici récupéré la chaine d'option pour NVDA pour une échéance donnée. Cela nous permettra de réaliser un backtest, pour cela on a besoin aussi des prix spots, ainsi que du taux sans risque, du taux de dividende